In [69]:
import numpy as np 
import pandas as pd 
import cv2 
import os 
import glob
import tqdm
from random import shuffle
import pickle
from os.path import basename
import imutils

In [2]:
path="/home/geekysethi/Desktop/vrinda_project/Dataset"

path_dir=np.sort(glob.glob(path+'/*'))


In [3]:
# print(path_dir)
all_csv=[]
for i in path_dir:
	
	all_csv.append(glob.glob(str(i)+'/*.csv'))

# print(all_csv)


In [65]:
def extract_color_histogram(image, bins=(8, 8, 8)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
		[0, 180, 0, 256, 0, 256])
 
	# handle normalizing the histogram if we are using OpenCV 2.4.X
	if imutils.is_cv2():
		hist = cv2.normalize(hist)
 
	# otherwise, perform "in place" normalization in OpenCV 3 (I
	# personally hate the way this is done
	else:
		cv2.normalize(hist, hist)
 
	# return the flattened histogram as the feature vector
	return hist.flatten()

In [70]:
train_images=[]
labels=[]
for j in all_csv:
    df=pd.read_csv(str(j)[2:-2])

    for i in range(len(df)):

#         print(i)
        image_path=os.path.join(path,df.image[i])
        img=cv2.imread(image_path,1)
        img=img[df.top_left_y[i]:df.bottom_right_y[i],df.top_left_x[i]:df.bottom_right_x[i]]
#         img=cv2.resize(img,(50,50)).flatten()
        img=extract_color_histogram(img)
        
        label=basename(os.path.join(path,df.image[i]))[0]

        train_images.append(img)
        labels.append(label)
        # cv2.imshow('frame',img)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()



# for j in all_csv:
# 	print(str(j)[2:-2])
# 	df=pd.read_csv(str(j)[2:-2])
# 	print(df.head())


In [71]:
print(len(train_images))
print(len(labels))
print(np.shape(train_images))

1680
1680
(1680, 512)


In [72]:
print(np.shape(train_images))
print(np.shape(labels))

(1680, 512)
(1680,)


In [73]:
# print(train_images[0])
print(type(labels))

# print(labels[:300])
print(np.unique(labels))

<class 'list'>
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S'
 'T' 'U' 'V' 'W' 'X' 'Y']


In [74]:
from sklearn import preprocessing

enocoded_lables= preprocessing.LabelEncoder()
enocoded_lables.fit(labels)
new_lables=enocoded_lables.transform(labels)

In [75]:
print(np.unique(new_lables))
total_lables=len(np.unique(new_lables))
print(total_lables)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
24


In [76]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_images,new_lables,test_size=0.2)

In [77]:
len(X_train)

1344

In [78]:
# print(X_train[:12])
print(np.shape(X_train))

(1344, 512)


In [86]:
from sklearn.neighbors import KNeighborsClassifier

classifier=KNeighborsClassifier(n_neighbors=5)

In [87]:
classifier.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [88]:
classifier.score(X_test,y_test)

0.7232142857142857

In [83]:
for i in range(1,24):
    classifier=KNeighborsClassifier(n_neighbors=i)
    classifier.fit(X_train,y_train)
    print(classifier.score(X_test,y_test))
    

0.922619047619
0.869047619048
0.830357142857
0.755952380952
0.723214285714
0.630952380952
0.574404761905
0.505952380952
0.470238095238
0.431547619048
0.401785714286
0.363095238095
0.339285714286
0.330357142857
0.291666666667
0.267857142857
0.255952380952
0.247023809524
0.238095238095
0.238095238095
0.22619047619
0.205357142857
0.202380952381


In [84]:
from sklearn import svm
clf=svm.SVC(decision_function_shape='ovo')
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.020833333333333332

In [85]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.14583333333333334

In [92]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=24, random_state=0).fit(X_train)

In [94]:
kmeans.score(X_test)

-74.802361311679221

In [95]:
kmeans.labels_

array([ 8, 20,  2, ...,  4,  7,  9], dtype=int32)